In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv("./Airbnb-US-2023.csv")
# Give some columns clearer/shorter names
df = df.rename(columns={
    "price": "price_per_night",
    "number_of_reviews": "nr_of_reviews",
    "last_review": "last_review_date",
    "calculated_host_listings_count": "nr_of_listings_by_host",
    "availability_365": "days_per_year_available",
    "number_of_reviews_ltm": "nr_of_reviews_last_twelve_months"
})
df.head()

## Visualizing the locations
### Use the output of the function below with google my maps to plot a number of locations unto a map

In [ ]:
def export_locations_to_google_my_maps(dataframe, number_of_locations = 10000):
    """Extracts latitude, longitude, name, and room_type from the dataframe provided.
    The locations are reduced until their number is close to number_of_locations.
    The result is grouped based on room_type, each group is exported to multiple csv files of max length = 2000.
    These are to be imported on Google My Maps for plotting."""
    
    dataframe = dataframe[["latitude", "longitude", "name", "room_type"]]
    
    entire_apartment_df = dataframe[dataframe["room_type"] == "Entire home/apt"].drop("room_type", axis=1)
    private_room_df = dataframe[dataframe["room_type"] == "Private room"].drop("room_type", axis=1)
    shared_room_df = dataframe[dataframe["room_type"] == "Shared room"].drop("room_type", axis=1)
    hotel_room_df = dataframe[dataframe["room_type"] == "Hotel room"].drop("room_type", axis=1)
    
    locations_to_eliminate = len(dataframe) - number_of_locations
    entire_apartment_ratio = len(entire_apartment_df) / len(dataframe)
    private_room_ratio = len(private_room_df) / len(dataframe)
    shared_room_ratio = len(shared_room_df) / len(dataframe)
    hotel_room_ratio = len(hotel_room_df) / len(dataframe)
    
    entire_apartment_locations_to_delete = int(entire_apartment_ratio * locations_to_eliminate)
    private_room_locations_to_delete = int(private_room_ratio * locations_to_eliminate)
    shared_room_locations_to_delete = int(shared_room_ratio * locations_to_eliminate)
    hotel_room_locations_to_delete = int(hotel_room_ratio * locations_to_eliminate)
    
    entire_apartment_locations_to_keep = len(entire_apartment_df) - entire_apartment_locations_to_delete
    private_room_locations_to_keep = len(private_room_df) - private_room_locations_to_delete
    shared_room_locations_to_keep = len(shared_room_df) - shared_room_locations_to_delete
    hotel_room_locations_to_keep = len(hotel_room_df) - hotel_room_locations_to_delete
    
    entire_apartment_df_shortened = entire_apartment_df.sample(entire_apartment_locations_to_keep)
    private_room_df_shortened = private_room_df.sample(private_room_locations_to_keep)
    shared_room_df_shortened = shared_room_df.sample(shared_room_locations_to_keep)
    hotel_room_df_shortened = hotel_room_df.sample(hotel_room_locations_to_keep)
    
    entire_apartment_dfs = np.array_split(entire_apartment_df_shortened, len(entire_apartment_df_shortened) // 2000 + 1)
    for i, df_chunk in enumerate(entire_apartment_dfs):
        df_chunk.to_csv(f"google-my-maps-csvs/apartment{i}.csv")
    
    private_room_dfs = np.array_split(private_room_df_shortened, len(private_room_df_shortened) // 2000 + 1)
    for i, df_chunk in enumerate(private_room_dfs):
        df_chunk.to_csv(f"google-my-maps-csvs/private{i}.csv")
    
    shared_room_dfs = np.array_split(shared_room_df_shortened, len(shared_room_df_shortened) // 2000 + 1)
    for i, df_chunk in enumerate(shared_room_dfs):
        df_chunk.to_csv(f"google-my-maps-csvs/shared{i}.csv")
    
    hotel_room_dfs = np.array_split(hotel_room_df_shortened, len(hotel_room_df_shortened) // 2000 + 1)
    for i, df_chunk in enumerate(hotel_room_dfs):
        df_chunk.to_csv(f"google-my-maps-csvs/hotel{i}.csv")

## Cleaning the data
### Removing duplicates and irrelevant columns

In [ ]:
print(f"Length before removing irrelevant columns: {len(df)}")
relevant_df = df.drop(["id"], axis=1)
relevant_df = relevant_df.drop_duplicates()
# We drop neighbourhood_group because we already have the neighbourhood column which provides more information
relevant_df = relevant_df.drop(["name", "host_id", "host_name", "latitude", "longitude", "neighbourhood_group"], axis=1)
print(f"Length after removing irrelevant columns: {len(relevant_df)}")

### Dealing with missing values and outliers

In [ ]:
relevant_df.isnull().any()

In [ ]:
# The last_review_date values will be converted to unix timestamps
# Since the timestamp being 0 isn't accurate, a has_reviews column will be added to the dataset to handle this scenario
non_null_df = relevant_df.copy()
non_null_df["has_reviews"] = non_null_df.last_review_date.notnull().astype(int)
non_null_df.last_review_date = non_null_df.last_review_date.fillna(0)
non_null_df.reviews_per_month = non_null_df.reviews_per_month.fillna(0)
non_null_df.loc[non_null_df.has_reviews == 1, "last_review_date"] = pd.to_datetime(non_null_df[non_null_df.has_reviews == 1].last_review_date).apply(lambda x: int(x.timestamp()))

In [ ]:
# Create a histogram of every column that could have outliers to see which ones have outliers
# Alongside there will be plotted 2 vertical lines representing the bounds for eliminating outliers
columns_used_for_checking_outliers = non_null_df.drop(["neighbourhood", "room_type", "city", "has_reviews"], axis=1).columns

rows_of_subplots = int(len(columns_used_for_checking_outliers) / 2)
columns_of_subplots = 2

fig, axes = plt.subplots(rows_of_subplots, columns_of_subplots, figsize=(14, 10))
fig.subplots_adjust(hspace=0.9, wspace=0.2)
axes = axes.flatten()

for subplot_index, column_name in enumerate(columns_used_for_checking_outliers):
    ax = axes[subplot_index]
    ax.hist(non_null_df[column_name], bins=20, rwidth=0.8)
    
    mean = non_null_df[column_name].mean()
    standard_deviation = non_null_df[column_name].std()
    
    lower_bound = mean - (3 * standard_deviation)
    upper_bound = mean + (3 * standard_deviation)

    ax.axvline(x=lower_bound, color='b')
    ax.axvline(x=upper_bound, color='b')
    
    # most subplots need to be scaled logarithmically because there is one range of values that is much more frequent
    # than any other, thus the subplot won't give too much information unless it is scaled
    if column_name != "days_per_year_available":
        ax.set_yscale("log")
    
    ax.set_xlabel(column_name)
    ax.set_ylabel("frequency")
    ax.set_title(f"Distribution of {column_name}")
    
plt.show()

In [ ]:
# last_review_date and days_per_year_available will be excluded from outlier removal because they don't have any outliers

columns_used_for_removing_outliers = non_null_df.drop(["neighbourhood", "room_type", "city", "has_reviews", "last_review_date", "days_per_year_available"], axis=1).columns

print(f"Length before removing outliers: {len(non_null_df)}\n")

no_outliers_df = non_null_df.copy()

for column_name in columns_used_for_removing_outliers:
    mean = no_outliers_df[column_name].mean()
    standard_deviation = no_outliers_df[column_name].std()
    
    lower_bound = mean - (3 * standard_deviation)
    upper_bound = mean + (3 * standard_deviation)
    
    percentage_removed = round((((no_outliers_df[column_name] < lower_bound) | (no_outliers_df[column_name] > upper_bound)).sum() / len(no_outliers_df)) * 100, 2)

    print(f"For column {column_name}, removing a percentage of {percentage_removed}% values.")
    no_outliers_df = no_outliers_df[(lower_bound <= no_outliers_df[column_name]) & (no_outliers_df[column_name] <= upper_bound)]

print(f"\nLength after removing outliers: {len(no_outliers_df)}")